动态网页的爬取

In [51]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import json
import re
import multiprocessing
# import xlwt
import time
import pandas as pd

In [3]:
# 手动找到真实的request headers

wangyi = 'https://news.163.com/special/cm_yaowen20200213_04/?callback=data_callback'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.55'}
response = requests.get(wangyi, headers=header)
print(response.status_code)

200


In [9]:
# data = response.text
print(response.text)

data_callback([
                        
                        
                                                            
                                                                     {
        "title":"“9·11”纪念日当天拜登喊话美军：我们欠你们的",
        "digest":"",
        "docurl":"https://www.163.com/dy/article/HH182KHJ0514D3UH.html",
        "commenturl":"https://comment.tie.163.com/HH182KHJ0514D3UH.html",
        "tienum":1169,
        "tlastid":"<a href='https://news.163.com' class='link link_more'><em>新闻</em></a>",
        "tlink":"https://www.163.com/dy/article/HH182KHJ0514D3UH.html",
        "label":"其它",
        "keywords":[
                                                ],
        "time":"09/12/2022 08:22:31",
        "newstype":"article",
        "pics3":[
                    ],
        "channelname":"yaowen",
        "source":"封面新闻",
        "point":"70",
                    "imgurl":"http://cms-bucket.ws.126.net/2022/0912/cb9e7e73p00ri2m7n005ec000hs00fcc.png",
            "a

# 逐步讲解

In [20]:
# response.text 不符合json格式，去掉最外层（）保留为列表

start = response.text.index('[')
end = response.text.index('])') + 1
print(start, end)

14 80314


In [18]:
print(response.text[start:end])

[
                        
                        
                                                            
                                                                     {
        "title":"“9·11”纪念日当天拜登喊话美军：我们欠你们的",
        "digest":"",
        "docurl":"https://www.163.com/dy/article/HH182KHJ0514D3UH.html",
        "commenturl":"https://comment.tie.163.com/HH182KHJ0514D3UH.html",
        "tienum":1169,
        "tlastid":"<a href='https://news.163.com' class='link link_more'><em>新闻</em></a>",
        "tlink":"https://www.163.com/dy/article/HH182KHJ0514D3UH.html",
        "label":"其它",
        "keywords":[
                                                ],
        "time":"09/12/2022 08:22:31",
        "newstype":"article",
        "pics3":[
                    ],
        "channelname":"yaowen",
        "source":"封面新闻",
        "point":"70",
                    "imgurl":"http://cms-bucket.ws.126.net/2022/0912/cb9e7e73p00ri2m7n005ec000hs00fcc.png",
            "add1":"",
     

In [17]:
# 使用json转换成python数据格式

data = json.loads(response.text[start:end])
print(data)

[{'title': '“9·11”纪念日当天拜登喊话美军：我们欠你们的', 'digest': '', 'docurl': 'https://www.163.com/dy/article/HH182KHJ0514D3UH.html', 'commenturl': 'https://comment.tie.163.com/HH182KHJ0514D3UH.html', 'tienum': 1169, 'tlastid': "<a href='https://news.163.com' class='link link_more'><em>新闻</em></a>", 'tlink': 'https://www.163.com/dy/article/HH182KHJ0514D3UH.html', 'label': '其它', 'keywords': [], 'time': '09/12/2022 08:22:31', 'newstype': 'article', 'pics3': [], 'channelname': 'yaowen', 'source': '封面新闻', 'point': '70', 'imgurl': 'http://cms-bucket.ws.126.net/2022/0912/cb9e7e73p00ri2m7n005ec000hs00fcc.png', 'add1': '', 'add2': '', 'add3': ''}, {'title': '英国示威者抗议黑人歌手遭枪击 英媒：他们来悼念女王', 'digest': '', 'docurl': 'https://www.163.com/dy/article/HH10BSLG05504DPG.html', 'commenturl': 'https://comment.tie.163.com/HH10BSLG05504DPG.html', 'tienum': 913, 'tlastid': "<a href='https://news.163.com' class='link link_more'><em>新闻</em></a>", 'tlink': 'https://www.163.com/dy/article/HH10BSLG05504DPG.html', 'label': '其它', 'k

In [56]:
# 数据清洗 + 二次爬取（点击链接，爬取新闻内容）
cnt = 0
for item in data:
    # print(item['title'])
    # print(item['time'])
    cnt += 1
    # print(item['docurl'])
    # print(item['source'])
    # inurl = item['docurl']
    # print(inurl)
    # inresponse = requests.get(inurl, headers=header).text

    # Xpath 不适合
    # inhtml = etree.HTML(inresponse)
    # print(inhtml.xpath('//div[@class="post_body"]/text()'))

    # BeautifulSoup
    # bs = BeautifulSoup(inresponse, 'lxml')
    # news = bs.findAll('div', class_='post_body')[0].text
    # print(news)

    # break
print(cnt)
# data = json.loads(response.text)
# print(data)

70


# 合并
思路
1. 抓取每次翻页的页面链接 (每页有70)
2. 每一页各新闻的标题
3. 每一新闻url跳转后的内容
4. 整理合并

In [53]:
# https://news.163.com/special/cm_yaowen20200213/?callback=data_callback
# https://news.163.com/special/cm_yaowen20200213_02/?callback=data_callback

def PageUrl(end, ls_title, ls_time, ls_doc, ls_source):
    ls_url = []
    for i in range(1, end+1):
        if i == 1:
            url = 'https://news.163.com/special/cm_yaowen20200213/?callback=data_callback'
        else:
            url = 'https://news.163.com/special/cm_yaowen20200213_0{}/?callback=data_callback'.format(str(i + 1))
        # ls_url.append(url)
        parse_web(url, ls_title, ls_time, ls_doc, ls_source)


def parse_web(url, ls_title, ls_time, ls_doc, ls_source):
    response = requests.get(url, headers=header)
    # JSON's need
    start = response.text.index('[')
    end = response.text.index('])') + 1
    data = json.loads(response.text[start:end])
    # parse
    for item in data:
        title = item['title']
        time = item['time']
        source = item['source']
        # inner url - news
        docurl = item['docurl']
        try:
            docresponse = requests.get(docurl, headers=header).text
            doc_bs = BeautifulSoup(docresponse, 'lxml')
            doc = doc_bs.findAll('div', class_='post_body')[0].text
        except:
            doc = 'empty'

        #此处也可以逐个写入
        ls_title.append(title)
        ls_time.append(time)
        ls_source.append(source)
        ls_doc.append(doc)

        # add sleep 反扒技巧
        # time.sleep(3)


    return ls_title, ls_time, ls_source, ls_doc


In [54]:

ls_title = []
ls_time = []
ls_source = []
ls_doc = []

PageUrl(2, ls_title, ls_time, ls_source, ls_doc)

In [55]:
# print(wangyi.split('/', 2))

dic = {'title': ls_title, 'time': ls_time, 'source': ls_source, 'doc': ls_doc}
df = pd.DataFrame(dic)
print(df)

                           title                 time  \
0      现场披露!乌军炮击命中俄军车辆 一名俄士兵奇迹生还                        
1      丈夫称妻子结婚3年5次出轨 岳母:你就不能多包容点  09/15/2022 22:48:51   
2     30岁美貌人妻嫌丈夫无趣 与新认识男子在车内发生关系  09/15/2022 22:35:04   
3      神秘的印钞集团又有干部落马 前任总经理被查后不收手  09/15/2022 22:17:27   
4      乌克兰对西方说话口气逐渐强硬 认识到了自己的价值?  09/15/2022 22:16:16   
..                           ...                  ...   
135       媒体：俄乌冲突200天 俄罗斯的损失有多大？  09/12/2022 11:38:28   
136     拉夫罗夫：西方制裁为＂消除强大的对手＂ 包括中国  09/12/2022 11:33:27   
137     美国正加大对乌军援 欧盟突然宣布：武器库里没货了  09/12/2022 11:31:17   
138    美元走强人民币被动贬值会出事吗?央行的表态很有底气  09/12/2022 11:17:56   
139  中纪委机关报:黑龙江19户＂国皮民骨＂涉粮企业完成整改  09/12/2022 10:22:23   

                                                source       doc  
0                                                empty      军情透视  
1    \n“现在的家对我来说就像一个逃不出去的牢笼！”22岁的彭冰冰面对媒体调解员时愤愤地说道。彭...       冷凝史  
2    \n相当自私的出轨案件！一女子抛夫弃子与小10岁的小男友同居并生子！北青网披露了一则出轨重婚...  七月生活情感驿站  
3    \n据中央纪委国家监委驻中国人民银行纪检监察组、河北省纪委监委昨日消息：中国印钞造币

In [57]:
df.to_excel('News.xlsx',index=False)